In [9]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from typing import *
from pprint import pprint

import bigbench.api.results as bb

from lass.log_handling import LogLoader, TaskLog
from lass.datasets import split_task_level, analyse, merge

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
loader = (LogLoader(logdir = Path('../artifacts/logs'))
        .with_tasks('paper-full')
        .with_model_families(['BIG-G T=0'])
        .with_model_sizes(['128b'])
        .with_shots([0])
        .with_query_types([bb.MultipleChoiceQuery])
)

train, test = split_task_level(loader, seed=42, test_fraction=0.2)

In [11]:
stats = merge(analyse(train), analyse(test), 'train', 'test')
del stats['task_names'] # Can't read anything anymore otherwise
pprint(stats)

{'metrics': {'lm-acc': {'test': 0.31445908841295994,
                        'train': 0.3419742693088115}},
 'stats': {'n_instances': {'test': 9105, 'train': 46326},
           'n_instances_nonbinary': {'test': 119, 'train': 254},
           'n_tasks': {'test': 23, 'train': 95}}}


In [3]:
n_queries = sum(len(log.queries or []) for log in tasks)
print(f"{n_queries} queries")

337 queries


In [4]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import torch

print(torch.cuda.device_count())

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
1


In [5]:
from sklearn.model_selection import train_test_split
import pandas as pd

train_tasks, test_tasks = train_test_split(tasks, test_size=0.2, random_state=42)
print(f"Train tasks: {[t.task.task_name for t in train_tasks]}")
print(f"Test tasks: {[t.task.task_name for t in test_tasks]}")

def to_dataframe(tasks: List[bb.ResultsFileData]) -> pd.DataFrame:
    dfs: List[pd.DataFrame] = []
    for task in tasks:
        for query in (task.queries or []):
            dfs.append(pd.DataFrame(query.samples))
    return pd.concat(dfs)

df_train = to_dataframe(train_tasks)
df_test = to_dataframe(test_tasks)

print("Original data:")
display(df_train.head(1))
display(df_train['correct'].value_counts().head(4))


Train tasks: ['symbol_interpretation', 'salient_translation_error_detection', 'minute_mysteries_qa', 'talkdown', 'dyck_languages', 'natural_instructions', 'language_identification', 'hinglish_toxicity', 'simp_turing_concept', 'analogical_similarity', 'misconceptions_russian', 'physics', 'suicide_risk', 'factuality_of_summary', 'international_phonetic_alphabet_nli', 'formal_fallacies_syllogisms_negation', 'conlang_translation', 'sufficient_information', 'qa_wikidata', 'movie_dialog_same_or_different', 'intersect_geometry', 'disfl_qa', 'logical_sequence', 'color', 'snarks', 'metaphor_understanding', 'human_organs_senses', 'unit_conversion', 'bias_from_probabilities', 'authorship_verification', 'crash_blossom', 'simple_ethical_questions', 'arithmetic', 'dark_humor_detection', 'matrixshapes', 'unqover', 'intent_recognition', 'bbq_lite_json', 'social_iqa', 'physics_questions', 'abstract_narrative_understanding', 'rephrase', 'polish_sequence_labeling', 'gre_reading_comprehension', 'english_p

,input,targets,scores,target_values,correct,absolute_scores,normalized_scores,metrics
0,\nIn the SIT-adversarial world a structure is ...,[There is at least one triangle pointing down....,"[-9.594904899597168, -7.830936431884766, -8.70...","{'There are at least two blue pieces. ': 0, 'T...",1.0,"[-9.594904899597168, -7.830936431884766, -8.70...","[-2.8301148414611816, -1.0661463737487793, -1....",{'calibration_multiple_choice_brier_score': 0....


0.0    30373
1.0    15699
0.8       85
0.6       63
Name: correct, dtype: int64

In [6]:
def huggingfaceify(df: pd.DataFrame) -> pd.DataFrame:
    df_hf = df[['input', 'correct']].rename(columns={'input':'text','correct':'label'})
    df_hf = df_hf[df_hf['label'].isin([0.0, 1.0])]
    df_hf[['label']] = df_hf[['label']].astype(int)
    return df_hf


print("\n\nHuggingface ready data:")
dfhf_train = huggingfaceify(df_train)
dfhf_test = huggingfaceify(df_test)
display(dfhf_train.head(1))
display(dfhf_train['label'].value_counts().head(5))




Huggingface ready data:


,text,label
0,\nIn the SIT-adversarial world a structure is ...,1


0    30373
1    15699
Name: label, dtype: int64

In [7]:
from datasets import Dataset, DatasetDict


display(f"Train accuracy: {dfhf_train['label'].mean():.2f} ({len(dfhf_train)} instances)")
display(f"Test accuracy: {dfhf_test['label'].mean():.2f} ({len(dfhf_test)} instances)")

ds = DatasetDict()
ds['train'] = Dataset.from_pandas(dfhf_train, split='train')
ds['test'] = Dataset.from_pandas(dfhf_test, split='test')

dataset = ds
ds['train'][0]

'Train accuracy: 0.34 (46072 instances)'

'Test accuracy: 0.31 (8986 instances)'

{'text': '\nIn the SIT-adversarial world a structure is a sequence of six emojis.\nHereafter are reported the emojis used along with their descriptions.\n 🔺 is a red circle;\n 🟦 is a blue circle;\n 🔴 is a yellow circle;\n 🟥 is a red triangle pointing up;\n 🟨 is a red triangle pointing down;\n 🔻 is a red square;\n 🟡 is a blue square;\n _ is a yellow square;\n 🔵 is an empty space.\n\nChoose the sentence consistent with the structure 🟥 🔻 🔺 🟡 🟥 🟨 and not consistent with 🔺 🟡 🟡 🟨 🟦 _:\n\n  choice: There are at least two triangles.\n\n  choice: There is at least one triangle.\n\n  choice: There are at least two yellow squares.\n\n  choice: There are at least two blue pieces.\n\n  choice: There is at least one triangle pointing down.\n\nA: ',
 'label': 1,
 '__index_level_0__': 0}

In [8]:
# Analyse LM results
import math
import sklearn as sk
import numpy as np

lm_preds = df_test[df_test['correct'].isin([0.0, 1.0])].copy()

def confidence(row):
    return np.max(row['normalized_scores'])

# Assumes the actual output is arg_max
lm_preds['confidence_normalized'] = lm_preds.apply(lambda row: math.exp(np.max(row['normalized_scores'])), axis=1)
lm_preds['confidence_absolute'] = lm_preds.apply(lambda row: math.exp(np.max(row['absolute_scores'])), axis=1)

display(sk.metrics.roc_auc_score(lm_preds['correct'], lm_preds['confidence_normalized']))
display(sk.metrics.roc_auc_score(lm_preds['correct'], lm_preds['confidence_absolute']))

0.5903410113824089

0.5514382335446202

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt")
    return tokenizer(examples["text"], padding="max_length", truncation=True)
    # return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=1024) # longformer
    # return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="np") #gpt-2
    # return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=2048) # xlnet

# tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

  0%|          | 0/47 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [10]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42) #.select(range(50))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42) #.select(range(50))
len(train_dataset), len(eval_dataset)

(46072, 8986)

In [11]:
import wandb
%env WANDB_PROJECT=lass
%env WANDB_LOG_MODEL=true
%env TOKENIZERS_PARALLELISM=true
wandb.login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
env: WANDB_PROJECT=lass
env:

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: wschella (use `wandb login --relogin` to force relogin)


True

In [12]:
from transformers.models.auto.modeling_auto import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import BertModel, BertConfig
import numpy as np
from datasets import load_metric
import scipy
import torch.nn as nn

# model = BertModel(BertConfig.from_pretrained("bert-base-cased"))
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2")
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=2)
# model.config.pad_token_id = model.config.eos_token_id
# model = AutoModelForSequenceClassification.from_pretrained("./test_trainer/checkpoint-13500", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("../artifacts/assessors/bert-bs32/checkpoint-3000", num_labels=2)

training_args = TrainingArguments(
    output_dir="albert-base-v2-bs32-task-split",
    run_name="albert-base-v2-bs32-task-split",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    report_to="wandb",
    num_train_epochs=3,
    # max_steps=17277 - 3000,
)

metrics = {
    "accuracy": load_metric("accuracy"),
    "precision": load_metric("precision"),
    "recall": load_metric("recall"),
    "f1": load_metric("f1"),
    "roc_auc": load_metric("roc_auc"),
}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)    
    # ROC AUC metric requires probabilities instead of logits, and only of the "postive" class (="highest label" = 1).
    # Needs to change for multi-class or multi-label.
    prediction_scores = scipy.special.softmax(logits,axis=-1)[:,-1]
    return {
          "accuracy": metrics["accuracy"].compute(predictions=predictions, references=labels)["accuracy"],
          "precision": metrics["precision"].compute(predictions=predictions, references=labels)["precision"],
          "recall": metrics["recall"].compute(predictions=predictions, references=labels)["recall"],
          "f1": metrics["f1"].compute(predictions=predictions, references=labels)["f1"],
          "roc_auc": metrics["roc_auc"].compute(prediction_scores=prediction_scores, references=labels)["roc_auc"],
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

Downloading:   0%|          | 0.00/45.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [13]:
torch.cuda.empty_cache()
trainer.train()

wandb.finish()

The following columns in the training set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
/home/wout/pp/lass/.env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 46072
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4320
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
1,0.631900,0.573639,0.687180,0.000000,0.000000,0.000000,0.666050
2,0.584800,0.566076,0.714667,0.581411,0.313767,0.407579,0.690441
3,0.558200,0.568525,0.702203,0.538726,0.334045,0.412385,0.694846


Saving model checkpoint to albert-base-v2-bs32-task-split/checkpoint-500
Configuration saved in albert-base-v2-bs32-task-split/checkpoint-500/config.json
Model weights saved in albert-base-v2-bs32-task-split/checkpoint-500/pytorch_model.bin
Saving model checkpoint to albert-base-v2-bs32-task-split/checkpoint-1000
Configuration saved in albert-base-v2-bs32-task-split/checkpoint-1000/config.json
Model weights saved in albert-base-v2-bs32-task-split/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8986
  Batch size = 32
/home/wout/pp/lass/.env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined a

eval/accuracy,▁█▅
eval/f1,▁██
eval/loss,█▁▃
eval/precision,▁█▇
eval/recall,▁██
eval/roc_auc,▁▇█
eval/runtime,▁▁█
eval/samples_per_second,██▁
eval/steps_per_second,██▁
train/epoch,▁▂▃▃▄▅▅▆▆▇██
train/global_step,▁▂▃▃▄▅▅▆▆▇██


In [14]:
import wandb
# wandb.finish()